# 0. read in data

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import db_dtypes
from plotly import tools
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import geopandas as gpd
from shapely.wkt import loads

In [119]:
# Loading Data
food_license = pd.read_csv(
    'data_outputs/food_licenses_cleaned.csv',
    dtype={'zip_code': str},  # force zip_code column to be string
    header='infer'
)

divvy_stations = pd.read_csv('data_outputs/divvy_stations_cleaned.csv', header='infer')
population_counts = pd.read_csv('data_outputs/population_counts_cleaned.csv', header='infer')
yearly_average = pd.read_csv('data_exports/yearly_average.csv', header='infer')
zoning_data = pd.read_csv('data_outputs/zoning_data_cleaned.csv', header='infer')
bus_stop = pd.read_csv('data_outputs/bus_stations_cleaned.csv', header='infer')
food_license.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,geometry,food_category,location_score
0,3025375,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.67793789...",POINT (-87.6779378973854 41.896004699879654),event_food,51
1,3025642,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,60622,"{""type"": ""Point"", ""coordinates"": [-87.67843725...",POINT (-87.67843725836676 41.903059772046284),event_food,51
2,3020896,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,-87.592374,CHICAGO,1400 E 47TH DR E,60653,"{""type"": ""Point"", ""coordinates"": [-87.59237386...",POINT (-87.59237386663331 41.80979548271692),restaurant,51
3,3020976,SAM 95 FOOD INC.,SAM 95 FOOD INC.,767.0 | 775.0,Retail Food Establishment,Retail Sales of General Merchandise and Non-Pe...,41.721357,-87.659962,CHICAGO,1452 W 95TH ST 1ST,60643,"{""type"": ""Point"", ""coordinates"": [-87.65996168...",POINT (-87.65996168027644 41.72135685967372),convenience_store,56
4,3020057,STALWART MARKET,STALWART MARKET LLC,1132.0,Mobile Food License,Sales of Produce From a Produce Stand Located ...,41.732108,-87.543374,CHICAGO,3320-3322 E 90TH ST,60617,"{""type"": ""Point"", ""coordinates"": [-87.54337357...",POINT (-87.5433735775229 41.73210827910671),food_truck,66


In [114]:
food_license.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,food_category,location_score,is_food,is_bus_stop,is_divvy_station,geometry
0,3025375,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.67793789...",event_food,35,1,0,0,POINT (-87.6779378973854 41.896004699879654)
1,3025642,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,60622,"{""type"": ""Point"", ""coordinates"": [-87.67843725...",event_food,58,1,0,0,POINT (-87.67843725836676 41.903059772046284)
2,3020896,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,-87.592374,CHICAGO,1400 E 47TH DR E,60653,"{""type"": ""Point"", ""coordinates"": [-87.59237386...",restaurant,53,1,0,0,POINT (-87.59237386663331 41.80979548271692)
3,3020976,SAM 95 FOOD INC.,SAM 95 FOOD INC.,767.0 | 775.0,Retail Food Establishment,Retail Sales of General Merchandise and Non-Pe...,41.721357,-87.659962,CHICAGO,1452 W 95TH ST 1ST,60643,"{""type"": ""Point"", ""coordinates"": [-87.65996168...",convenience_store,98,1,0,0,POINT (-87.65996168027644 41.72135685967372)
4,3020057,STALWART MARKET,STALWART MARKET LLC,1132.0,Mobile Food License,Sales of Produce From a Produce Stand Located ...,41.732108,-87.543374,CHICAGO,3320-3322 E 90TH ST,60617,"{""type"": ""Point"", ""coordinates"": [-87.54337357...",food_truck,40,1,0,0,POINT (-87.5433735775229 41.73210827910671)


In [120]:
bus_stop.head()

,longitude,latitude,bus_stop_id,street,cross_st,city,public_nam,geometry
0,-87.747509,41.956892,15189.0,CICERO,BERTEAU (north leg),CHICAGO,Cicero & Berteau,POINT(-87.74750923800002 41.956892084)
1,-87.703980,41.805777,3082.0,KEDZIE,48TH PLACE (north leg),CHICAGO,Kedzie & 48th Place,POINT(-87.70398 41.805777001000024)
2,-87.640475,41.943530,1157.0,LAKE SHORE,ROSCOE,CHICAGO,Lake Shore & Roscoe,POINT(-87.64047498399998 41.943529618000014)
3,-87.585295,41.722579,7211.0,STONY ISLAND,95TH STREET,CHICAGO,Stony Island & 95th Street,POINT(-87.58529500100002 41.722579000999986)
4,-87.743174,41.802234,8123.0,CICERO,50TH STREET,CHICAGO,Cicero & 50th Street,POINT(-87.743173716 41.80223378400001)


In [121]:
divvy_stations.head()

,divvy_station_id,station_name,total_docks,docks_in_service,status,latitude,longitude,location_type,location_coordinates,geometry
0,1859717767703464460,Kedzie Ave & Redfield Dr,15,15,In Service,41.765230,-87.703160,Point,"[-87.70316, 41.76523]",POINT(-87.70316 41.76523)
1,a3a3dc37-a135-11e9-9cda-0a87ae2ba916,Fairbanks Ct & Grand Ave,15,15,In Service,41.891847,-87.620580,Point,"[-87.620580196381, 41.89184737211]",POINT(-87.6205801964 41.8918473721)
2,1861588043429399408,McCormick Blvd & Devon Ave,11,11,In Service,41.997200,-87.711050,Point,"[-87.71105, 41.9972]",POINT(-87.71105 41.9972)
3,2057127898206728832,Halsted St & Roscoe St,12,12,In Service,41.943670,-87.648950,Point,"[-87.64895, 41.94367]",POINT(-87.64895 41.94367)
4,9e1fbaaa-e872-4505-ada6-5080e62bef77,Lincoln Park Conservatory,31,31,In Service,41.923931,-87.635825,Point,"[-87.635824531317, 41.923931311366]",POINT(-87.6358245313 41.9239313114)


In [11]:
bus_stop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10760 entries, 0 to 10759
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   longitude   10760 non-null  float64
 1   latitude    10760 non-null  float64
 2   street      10760 non-null  object 
 3   cross_st    10760 non-null  object 
 4   city        10759 non-null  object 
 5   public_nam  10760 non-null  object 
 6   location    10760 non-null  object 
dtypes: float64(2), object(5)
memory usage: 588.6+ KB


# Draft of merge script

In [174]:
from shapely.wkt import loads as wkt_loads
import geopandas as gpd
from shapely.geometry import shape

food_license = pd.read_csv(
    'data_outputs/food_licenses_cleaned.csv',
    dtype={'zip_code': str},  # force zip_code column to be string
    header='infer'
)

divvy_stations = pd.read_csv('data_outputs/divvy_stations_cleaned.csv',dtype={'bus_stop_id': str}, header='infer')
population_counts = pd.read_csv('data_outputs/population_counts_cleaned.csv', header='infer')
yearly_average = pd.read_csv('data_exports/yearly_average.csv', header='infer')
zoning_data = pd.read_csv('data_outputs/zoning_data_cleaned.csv', header='infer')
bus_stop = pd.read_csv('data_outputs/bus_stations_cleaned.csv', header='infer')
# Add boolean col (this is help in looker filtering food/bus_stop/divvy_station if needed)
food_license['is_food'] = 1
food_license['is_bus_stop'] = 0
food_license['is_divvy_station'] = 0

bus_stop['is_food'] = 0
bus_stop['is_bus_stop'] = 1
bus_stop['is_divvy_station'] = 0

divvy_stations['is_food'] = 0
divvy_stations['is_bus_stop'] = 0
divvy_stations['is_divvy_station'] = 1
divvy_stations_v2 = divvy_stations.drop(['total_docks','status','location_type','location_coordinates'],axis=1)

# Append point type data sources together
combined_df = pd.concat([food_license, bus_stop,divvy_stations_v2], ignore_index=True)

# Convert WKT string to shapely geometry
combined_df_v2['geometry'] = combined_df['geometry'].apply(wkt_loads)

# Now create GeoDataFrame
# import geopandas as gpd
combined_gdf_v2 = gpd.GeoDataFrame(combined_df_v2, geometry='geometry', crs='EPSG:4326')
# combined_gdf.head()
zipcode = gpd.read_file('tl_2024_us_zcta520/tl_2024_us_zcta520.shp')
zipcode = zipcode.to_crs('EPSG:4326')

gdf_with_zip = gpd.sjoin(combined_gdf_v2, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')
gdf_with_zip = gdf_with_zip.rename(columns={'ZCTA5CE20': 'zip_code_2'})

gdf_with_zip['zip_code_2'] = gdf_with_zip['zip_code_2'].astype(str)
population_counts['zip_code'] = population_counts['zip_code'].astype(str)

gdf_with_zip_v2 = gdf_with_zip.merge(
    population_counts,
    how='left',
    left_on='zip_code_2',
    right_on='zip_code'
)

zoning_data = zoning_data.rename(columns={'index_right': 'zoning_index'})
# First convert string to dict
zoning_data['geometry'] = zoning_data['geometry'].apply(json.loads)

# Then convert dict to shapely geometry
zoning_data['geometry'] = zoning_data['geometry'].apply(shape)

# Now convert to GeoDataFrame
zoning_data = gpd.GeoDataFrame(zoning_data, geometry='geometry')

# Set CRS if known — guessing EPSG:4326
zoning_data.set_crs("EPSG:4326", inplace=True)

if 'index_right' in gdf_with_zip_v2.columns:
    gdf_with_zip_v2 = gdf_with_zip_v2.drop(columns='index_right')

# Perform spatial join: points (left) within polygons (right)
gdf_with_zip_v3 = gpd.sjoin(
    gdf_with_zip_v2,      # left: points
    zoning_data,          # right: polygons
    how='left',           # keep all points
    predicate='within'    # or 'intersects' if appropriate
)


In [175]:
gdf_with_zip_v3.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code_x,location,geometry,food_category,location_score,is_food,is_bus_stop,is_divvy_station,bus_stop_id,street,cross_st,public_nam,divvy_station_id,station_name,docks_in_service,zip_code_2,year,zip_code_y,population_total,population_0_to_17,population_18_to_29,population_30_to_39,population_40_to_49,population_50_to_59,population_60_to_69,population_70_to_79,population_80,population_female,population_male,population_latinx,population_asian,population_black,population_white,population_other,index_right,zoning_id,zone_class,edit_date,shape_area,shape_len,objectid,description,district_title,zoning_code_section,floor_area_ratio,maximum_building_height,lot_area_per_unit,front_yard_setback,side_setback,rear_yard_setback,rear_yard_open_space,on_site_open_space,minimum_lot_area,restaurant_allowed
0,3025375.0,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.6779378973854, 41.896004699879654]}",POINT (-87.67794 41.896),event_food,51.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60622,2021.0,60622,54650.0,7711.0,14330.0,17062.0,6684.0,3977.0,2478.0,1426.0,982.0,27177.0,27473.0,11270.0,2605.0,3189.0,35064.0,2522.0,3828.0,13.0,B3-2,2023-04-26 00:00:00+00:00,1.008442e+06,10187.052616,1918662.0,"Shopping centers, large stores, and retail storefronts, often along major streets. Allows more types of businesses than B1 and B2 districts. Apartments permitted above the ground floor.",Community Shopping District,17-3-0104,2.2,"Varies by lot frontage, and whether building has ground-floor commercial space. (See 17-3-0408)","1,000 sq ft/dwelling unit, 700 sq ft/efficiency unit, 700 sq ft/SRO unit","None, unless property borders an R-zoned lot. Then the front setback must be at least 50% of the R lot's front setback. (See 17-3-0404.)","None, unless property borders an R-zoned lot. Then the R lot's front setback applies.","If property has dwelling units, minimum of 30 ft. If its rear property line borders the side property line of an R-zoned lot, the rear setback must equal the side setback of the R-zoned lot. If rear line borders the R lot's rear line, setback must be at least 16 ft.",NaN,NaN,NaN,1.0
1,3025642.0,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,60622,"{""type"": ""Point"", ""coordinates"": [-87.67843725836676, 41.903059772046284]}",POINT (-87.67844 41.90306),event_food,51.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60622,2021.0,60622,54650.0,7711.0,14330.0,17062.0,6684.0,3977.0,2478.0,1426.0,982.0,27177.0,27473.0,11270.0,2605.0,3189.0,35064.0,2522.0,3727.0,13.0,B3-2,2023-02-03 00:00:00+00:00,6.692131e+05,7348.483182,1912405.0,"Shopping centers, large stores, and retail storefronts, often along major streets. Allows more types of businesses than B1 and B2 districts. Apartments permitted above the ground floor.",Community Shopping District,17-3-0104,2.2,"Varies by lot frontage, and whether building has ground-floor commercial space. (See 17-3-0408)","1,000 sq ft/dwelling unit, 700 sq ft/efficiency unit, 700 sq ft/SRO unit","None, unless property borders an R-zoned lot. Then the front setback must be at least 50% of the R lot's front setback. (See 17-3-0404.)","None, unless property borders an R-zoned lot. Then the R lot's front setback applies.","If property has dwelling units, minimum of 30 ft. If its rear property line borders the side property line of an R-zoned lot, the rear setback must equal the side setback of the R-zoned lot. If rear line borders the R lot's rear line, setback must be at least 16 ft.",NaN,NaN,NaN,1.0
2,3020896.0,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,

In [138]:
combined_df['geometry'].head()

0     POINT (-87.6779378973854 41.896004699879654)
1    POINT (-87.67843725836676 41.903059772046284)
2     POINT (-87.59237386663331 41.80979548271692)
3     POINT (-87.65996168027644 41.72135685967372)
4      POINT (-87.5433735775229 41.73210827910671)
Name: geometry, dtype: object

In [164]:
from shapely.wkt import loads as wkt_loads

# Convert WKT string to shapely geometry
combined_df_v2['geometry'] = combined_df['geometry'].apply(wkt_loads)

# Now create GeoDataFrame
# import geopandas as gpd
combined_gdf_v2 = gpd.GeoDataFrame(combined_df_v2, geometry='geometry', crs='EPSG:4326')
# combined_gdf.head()
zipcode = gpd.read_file('tl_2024_us_zcta520/tl_2024_us_zcta520.shp')
zipcode = zipcode.to_crs('EPSG:4326')

gdf_with_zip = gpd.sjoin(combined_gdf_v2, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')
gdf_with_zip = gdf_with_zip.rename(columns={'ZCTA5CE20': 'zip_code_2'})

gdf_with_zip.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,geometry,food_category,location_score,is_food,is_bus_stop,is_divvy_station,bus_stop_id,street,cross_st,public_nam,divvy_station_id,station_name,docks_in_service,index_right,zip_code_2
0,3025375.0,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.6779378973854, 41.896004699879654]}",POINT (-87.67794 41.896),event_food,51.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30807,60622
1,3025642.0,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,60622,"{""type"": ""Point"", ""coordinates"": [-87.67843725836676, 41.903059772046284]}",POINT (-87.67844 41.90306),event_food,51.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30807,60622
2,3020896.0,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,-87.592374,CHICAGO,1400 E 47TH DR E,60653,"{""type"": ""Point"", ""coordinates"": [-87.59237386663331, 41.80979548271692]}",POINT (-87.59237 41.8098),restaurant,51.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30345,60653
3,3020976.0,SAM 95 FOOD INC.,SAM 95 FOOD INC.,767.0 | 775.0,Retail Food Establishment,Retail Sales of General Merchandise and Non-Perishable Food | Retail Sales of Perishable Foods,41.721357,-87.659962,CHICAGO,1452 W 95TH ST 1ST,60643,"{""type"": ""Point"", ""coordinates"": [-87.65996168027644, 41.72135685967372]}",POINT (-87.65996 41.72136),convenience_store,56.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31300,60643
4,3020057.0,STALWART MARKET,STALWART MARKET LLC,1132.0,Mobile Food License,Sales of Produce From a Produce Stand Located on an Urban Farm (Produce Merchant),41.732108,-87.543374,CHICAGO,3320-3322 E 90TH ST,60617,"{""type"": ""Point"", ""coordinates"": [-87.5433735775229, 41.73210827910671]}",POINT (-87.54337 41.73211),food_truck,66.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31249,60617


In [165]:
# check number of zipcodes matching in the same row
# (gdf_with_zip['zip_code'] == gdf_with_zip['zip_code_2']).value_counts()
(gdf_with_zip[gdf_with_zip['is_food'] == 1]['zip_code'] == gdf_with_zip[gdf_with_zip['is_food'] == 1]['zip_code_2']).value_counts()



True     11007
False      737
Name: count, dtype: int64

In [166]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content in each column

mismatched_rows = gdf_with_zip[
    (gdf_with_zip['is_food'] == 1) &
    (gdf_with_zip['zip_code'] != gdf_with_zip['zip_code_2'])
]

mismatched_rows


,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,geometry,food_category,location_score,is_food,is_bus_stop,is_divvy_station,bus_stop_id,street,cross_st,public_nam,divvy_station_id,station_name,docks_in_service,index_right,zip_code_2
35,3012203.0,STARBUCKS COFFEE #11719,STARBUCKS CORPORATION,1050.0,Retail Food Establishment,Preparation and Sale of Coffee and/or Drinks,41.879942,-87.647453,CHICAGO,116 S HALSTED ST,60661,"{""type"": ""Point"", ""coordinates"": [-87.64745274382807, 41.8799423230123]}",POINT (-87.64745 41.87994),cafe,67.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30691,60607
48,3014846.0,THE UNDERSTUDY COFFEE AND BOOKS,"THE UNDERSTUDY COFFEE AND BOOKS, LLC",781.0 | 904.0 | 1050.0,Retail Food Establishment,Sale of Food Prepared Onsite With Dining Area | Retail Sales of General Merchandise | Preparation and Sale of Coffee and/or Drinks,41.982613,-87.668426,CHICAGO,5531 N CLARK ST,60640,"{""type"": ""Point"", ""coordinates"": [-87.66842628885883, 41.982613033865064]}",POINT (-87.66843 41.98261),convenience_store,65.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30654,60660
68,3006551.0,KISHA'S KITCHEN,KISHA'S KITCHEN 2 LLC,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.684943,-87.644369,CHICAGO,857 W 115TH ST,60643,"{""type"": ""Point"", ""coordinates"": [-87.644368901148, 41.68494313726917]}",POINT (-87.64437 41.68494),restaurant,78.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30224,60628
75,3009913.0,HENRY CITGO,"HENRY OIL, INC.",736.0 | 775.0,Retail Food Establishment,"Preparation of Food, Coffee or Drinks | Retail Sales of Perishable Foods",41.744152,-87.585945,CHICAGO,8300 S STONY ISLAND AVE,60617,"{""type"": ""Point"", ""coordinates"": [-87.58594506695293, 41.744151639041974]}",POINT (-87.58595 41.74415),restaurant,46.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31258,60619
166,2994958.0,DOLLAR GENERAL STORE # 11565,"DG RETAIL, LLC",775.0,Retail Food Establishment,Retail Sales of Perishable Foods,41.751714,-87.559431,CHICAGO,2649 E 79TH ST 1ST,60649,"{""type"": ""Point"", ""coordinates"": [-87.55943061435629, 41.75171392238672]}",POINT (-87.55943 41.75171),restaurant,52.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31249,60617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11512,2991055.0,ROBEY HOTEL,"1600 N. MILWAUKEE VENTURES, LLC",735.0,Retail Food Establishment,Preparation of Food and Dining on Premises With Seating,41.910437,-87.677761,CHICAGO,1600-1618 N MILWAUKEE AVE 1-13,60647,"{""type"": ""Point"", ""coordinates"": [-87.67776102073448, 41.91043679855338]}",POINT (-87.67776 41.91044),restaurant,69.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30807,60622
11563,2960984.0,TOTAL HOLISTIC CREAMERY,TOTAL HOLISTIC CREAMERY INCORPORATED,1083.0,Retail Food Establishment,Expedited Restaurant with On-Premises Consumption,41.910603,-87.677018,CHICAGO,1948 W NORTH AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.67701812652801, 41.91060331921216]}",POINT (-87.67702 41.9106),restaurant,46.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30223,60647
11565,2957534.0,WALGREENS #1298,"BOND DRUG COMPANY OF ILLINOIS, LLC",775.0,Retail Food Establishment,Retail Sales of Perishable Foods,41.906657,-87.671871,CHICAGO,1372 N MILWAUKEE AVE,60642,"{""type"": ""Point"", ""coordinates"": [-87.67187112432266, 41.906657377179585]}",POINT (-87.67187 41.90666),restaurant,16.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30807,60622
11567,2962341.0,CHENGDU BISTRO,MIAN MASTER INC.,735.0,Retail Food Establishment,Preparation of Food and Dining on Premises With Seating,41.910277,-87.682861,CHICAGO,2211 W NORTH AVE,60647,"{""type"": ""Point"", ""coordinates"": [-87.68286119020127, 41.91027701064524]}",POINT (-87.68286 41.91028),fine_dining,40.0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30807,60622


In [151]:
gdf_with_zip.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 23502 entries, 0 to 23501
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   license_id              11744 non-null  float64 
 1   doing_business_as_name  11744 non-null  object  
 2   legal_name              11744 non-null  object  
 3   business_activity_id    11744 non-null  object  
 4   license_description     11744 non-null  object  
 5   business_activity       11744 non-null  object  
 6   latitude                23502 non-null  float64 
 7   longitude               23502 non-null  float64 
 8   city                    22503 non-null  object  
 9   address                 11744 non-null  object  
 10  zip_code                11744 non-null  object  
 11  location                11744 non-null  object  
 12  geometry                23502 non-null  geometry
 13  food_category           11744 non-null  object  
 14  location_score     

In [134]:
import json
import geopandas as gdp
from shapely.geometry import shape

In [ ]:
from shapely import wkt

# Convert WKT string to geometry
divvy_stations_v3 = divvy_stations_v2.copy()
divvy_stations_v3['geometry'] = divvy_stations_v2['location'].apply(wkt.loads)

# Create GeoDataFrame
divvy_gdf = gpd.GeoDataFrame(divvy_stations_v3, geometry='geometry', crs='EPSG:4326')

# divvy_gdf.head()




,divvy_station_id,station_name,docks_in_service,latitude,longitude,location,is_food,is_bus_stop,is_divvy_station,geometry
0,1859717767703464460,Kedzie Ave & Redfield Dr,15,41.765230,-87.703160,POINT(-87.70316 41.76523),0,0,1,POINT (-87.70316 41.76523)
1,a3a3dc37-a135-11e9-9cda-0a87ae2ba916,Fairbanks Ct & Grand Ave,15,41.891847,-87.620580,POINT(-87.6205801964 41.8918473721),0,0,1,POINT (-87.62058 41.89185)
2,1861588043429399408,McCormick Blvd & Devon Ave,11,41.997200,-87.711050,POINT(-87.71105 41.9972),0,0,1,POINT (-87.71105 41.9972)
3,2057127898206728832,Halsted St & Roscoe St,12,41.943670,-87.648950,POINT(-87.64895 41.94367),0,0,1,POINT (-87.64895 41.94367)
4,9e1fbaaa-e872-4505-ada6-5080e62bef77,Lincoln Park Conservatory,31,41.923931,-87.635825,POINT(-87.6358245313 41.9239313114),0,0,1,POINT (-87.63582 41.92393)


# Test mapping of zip codes

In [66]:
zipcode = gpd.read_file('tl_2024_us_zcta520/tl_2024_us_zcta520.shp')
zipcode = zipcode.to_crs('EPSG:4326')
zipcode.head()


,ZCTA5CE20,GEOID20,GEOIDFQ20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,47236,47236,860Z200US47236,B5,G6350,S,1029063,0,+39.1517426,-085.7252769,"POLYGON ((-85.7341 39.15597, -85.72794 39.1561..."
1,47870,47870,860Z200US47870,B5,G6350,S,8830,0,+39.3701518,-087.4735141,"POLYGON ((-87.47414 39.37016, -87.47409 39.370..."
2,47851,47851,860Z200US47851,B5,G6350,S,53326,0,+39.5735839,-087.2459559,"POLYGON ((-87.24769 39.5745, -87.24711 39.5744..."
3,47337,47337,860Z200US47337,B5,G6350,S,303089,0,+39.8027537,-085.4372850,"POLYGON ((-85.44357 39.80328, -85.44346 39.803..."
4,47435,47435,860Z200US47435,B5,G6350,S,13302,0,+39.2657557,-086.2951577,"POLYGON ((-86.29592 39.26547, -86.29592 39.266..."


In [103]:
# Spatial join: find ZIP code polygon containing each station point
gdf_with_zip = gpd.sjoin(divvy_gdf, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='within')
gdf_with_zip = gdf_with_zip.rename(columns={'ZCTA5CE20': 'zip_code'})

gdf_with_zip.head()


,divvy_station_id,station_name,docks_in_service,latitude,longitude,location,is_food,is_bus_stop,is_divvy_station,geometry,index_right,zip_code
0,1859717767703464460,Kedzie Ave & Redfield Dr,15,41.765230,-87.703160,POINT(-87.70316 41.76523),0,0,1,POINT (-87.70316 41.76523),30228,60629
1,a3a3dc37-a135-11e9-9cda-0a87ae2ba916,Fairbanks Ct & Grand Ave,15,41.891847,-87.620580,POINT(-87.6205801964 41.8918473721),0,0,1,POINT (-87.62058 41.89185),30436,60611
2,1861588043429399408,McCormick Blvd & Devon Ave,11,41.997200,-87.711050,POINT(-87.71105 41.9972),0,0,1,POINT (-87.71105 41.9972),30319,60659
3,2057127898206728832,Halsted St & Roscoe St,12,41.943670,-87.648950,POINT(-87.64895 41.94367),0,0,1,POINT (-87.64895 41.94367),30051,60657
4,9e1fbaaa-e872-4505-ada6-5080e62bef77,Lincoln Park Conservatory,31,41.923931,-87.635825,POINT(-87.6358245313 41.9239313114),0,0,1,POINT (-87.63582 41.92393),30050,60614


In [52]:
food_license.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,food_category,location_score,is_food,is_bus_stop,is_divvy_station
0,3025375,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,60622,"{""type"": ""Point"", ""coordinates"": [-87.67793789...",event_food,35,1,0,0
1,3025642,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,60622,"{""type"": ""Point"", ""coordinates"": [-87.67843725...",event_food,58,1,0,0
2,3020896,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,-87.592374,CHICAGO,1400 E 47TH DR E,60653,"{""type"": ""Point"", ""coordinates"": [-87.59237386...",restaurant,53,1,0,0
3,3020976,SAM 95 FOOD INC.,SAM 95 FOOD INC.,767.0 | 775.0,Retail Food Establishment,Retail Sales of General Merchandise and Non-Pe...,41.721357,-87.659962,CHICAGO,1452 W 95TH ST 1ST,60643,"{""type"": ""Point"", ""coordinates"": [-87.65996168...",convenience_store,98,1,0,0
4,3020057,STALWART MARKET,STALWART MARKET LLC,1132.0,Mobile Food License,Sales of Produce From a Produce Stand Located ...,41.732108,-87.543374,CHICAGO,3320-3322 E 90TH ST,60617,"{""type"": ""Point"", ""coordinates"": [-87.54337357...",food_truck,40,1,0,0


In [53]:
bus_stop.head()

,longitude,latitude,bus_stop_id,street,cross_st,city,public_nam,location,is_food,is_bus_stop,is_divvy_station
0,-87.747509,41.956892,15189.0,CICERO,BERTEAU (north leg),CHICAGO,Cicero & Berteau,POINT(-87.74750923800002 41.956892084),0,1,0
1,-87.703980,41.805777,3082.0,KEDZIE,48TH PLACE (north leg),CHICAGO,Kedzie & 48th Place,POINT(-87.70398 41.805777001000024),0,1,0
2,-87.640475,41.943530,1157.0,LAKE SHORE,ROSCOE,CHICAGO,Lake Shore & Roscoe,POINT(-87.64047498399998 41.943529618000014),0,1,0
3,-87.585295,41.722579,7211.0,STONY ISLAND,95TH STREET,CHICAGO,Stony Island & 95th Street,POINT(-87.58529500100002 41.722579000999986),0,1,0
4,-87.743174,41.802234,8123.0,CICERO,50TH STREET,CHICAGO,Cicero & 50th Street,POINT(-87.743173716 41.80223378400001),0,1,0


In [54]:
combined_df.head()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,...,is_food,is_bus_stop,is_divvy_station,bus_stop_id,street,cross_st,public_nam,divvy_station_id,station_name,docks_in_service
0,3025375.0,LAO PENG YOU,LAO PENG YOU LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.896005,-87.677938,CHICAGO,2020 W CHICAGO AVE,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3025642.0,CHILLS ITALIAN ICE,LEMONCHILLO LLC,946.0,Special Event Food,180 Day Multiple Events - Special Event Food,41.903060,-87.678437,CHICAGO,2029 W DIVISION ST 1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3020896.0,TACO PROS/EGG HOLIC,1400 LAKE PARK INC.,782.0,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,41.809795,-87.592374,CHICAGO,1400 E 47TH DR E,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3020976.0,SAM 95 FOOD INC.,SAM 95 FOOD INC.,767.0 | 775.0,Retail Food Establishment,Retail Sales of General Merchandise and Non-Pe...,41.721357,-87.659962,CHICAGO,1452 W 95TH ST 1ST,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3020057.0,STALWART MARKET,STALWART MARKET LLC,1132.0,Mobile Food License,Sales of Produce From a Produce Stand Located ...,41.732108,-87.543374,CHICAGO,3320-3322 E 90TH ST,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
combined_df.tail()

,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,...,is_food,is_bus_stop,is_divvy_station,bus_stop_id,street,cross_st,public_nam,divvy_station_id,station_name,docks_in_service
23497,NaN,NaN,NaN,NaN,NaN,NaN,41.931248,-87.644336,NaN,NaN,...,0,0,1,NaN,NaN,NaN,NaN,a3a80a1e-a135-11e9-9cda-0a87ae2ba916,Clark St & Drummond Pl,23.0
23498,NaN,NaN,NaN,NaN,NaN,NaN,41.921057,-87.672797,NaN,NaN,...,0,0,1,NaN,NaN,NaN,NaN,a3a83ba1-a135-11e9-9cda-0a87ae2ba916,Wood St & Webster Ave,19.0
23499,NaN,NaN,NaN,NaN,NaN,NaN,41.920082,-87.677855,NaN,NaN,...,0,0,1,NaN,NaN,NaN,NaN,a3aa5ed1-a135-11e9-9cda-0a87ae2ba916,Damen Ave & Charleston St,11.0
23500,NaN,NaN,NaN,NaN,NaN,NaN,41.700121,-87.612260,NaN,NaN,...,0,0,1,NaN,NaN,NaN,NaN,87c01e57-8a00-430d-9eb3-2abe51e535dd,Vernon Ave & 107th St,15.0
23501,NaN,NaN,NaN,NaN,NaN,NaN,41.752487,-87.643902,NaN,NaN,...,0,0,1,NaN,NaN,NaN,NaN,c3f1be96-07c2-4c65-8a1d-f2203de88495,Halsted St & 78th St,15.0


In [37]:
from summarytools import dfSummary
dfSummary(combined_df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,license_id[float64],Mean (sd) : 2968368.7 (36812.3)min < med < max:2245772.0 < 2967221.0 < 3036139.0IQR (CV) : 62912.8 (80.6),"11,744 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjMsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvZiW1igAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAbBJREFUeJzt3UGK2nAYxuF/OtOpnYUigpfoDTzEHLZH6MITeAN3gkhEO1I1KeneqTra6QvPszUfBvIjGMiHVdu2Bf53nz76BOAcj2cdBWeoquprKeWpXOdX27avpz4UKjeLdDQavfT7/eE18+v1elVV1fdTsQqVW3nqIp1MJq+DwWB3yWBd173pdDpcLpfd3Vio3F8X6Xg8/nnFaPez4SQPU0QQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEsGL09xq56nfNM3ncidC5SY7T/v9vnc8Hr/tdrsfpZRr3vB/k1B5985TZz6fD2ez2fPhcHgodyBUbrLztFqt3tx5ei8PU0QQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKhL/+F2pVVb1Sypd/czp8oH7TNI91XXfX+2KbzebP3Ha77S0Wi+dLZs/5zt8VnVDOvVym2AAAAABJRU5ErkJggg=="">","10,760(47.8%)"
2,doing_business_as_name[object],1. nan2. SUBWAY3. DUNKIN DONUTS4. CITGO5. SHELL6. 7-ELEVEN7. BP8. MCDONALD'S9. MCDONALDS10. Dunkin Donuts11. other,"10,760 (47.8%)116 (0.5%)95 (0.4%)31 (0.1%)29 (0.1%)25 (0.1%)25 (0.1%)22 (0.1%)21 (0.1%)21 (0.1%)11,359 (50.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjMsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvZiW1igAAAAlwSFlzAAAPYQAAD2EBqD+naQAAA/RJREFUeJzt3UFuWlcYgNH7LJQmE5CFxBLaPbCILDaLYBHdA4pgEKeZUN0q48rG9UeJzxkgT+A9PX3mPtD97eVyuQwoPCRHAbFREhsZsZERGxmxkREbGbGRERuZZYzxaYzxoTvk3fpxuVyebn0S92y13W4/r9frx1ufyP/d+Xz+uizLF8FdbzVD2+/3T5vN5vsrXueXdjqdPh4Oh8fj8ThXALFdaTUfZmi73e7btS/yTszbDV7BBwQyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYqPdYjT3a3WHvD+uz39jNXegzo2B9mv9u3md5tbwW5/HPTOD8HxmEF5p8SezqN/Zht9akumq+YPJId7aw5yu+jnK576NN+V7NjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM22oGXnz8b5uBN2RZOxsALGe9sZAy8kDHwQsZXH2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAy8kLEtnIyBFzLu2ciIjczDsix/LMvi/2mSvLP9Psb47e0PxXtnGSUjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiI43tzzHGX90hea8MvJCxjJIRGxkDL2QMvJCxjJIRGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTHwQsbACxnLKBmxkTHwQsbACxnLKBmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMV1FxnQVGcsoGbGRMV1FxnQVGcsoGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZAy9kDLyQsYySERsZAy9kDLyQsYySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZEx8ELGwAsZyygZsZERGxnTVWRMV5GxjJIRGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTFdRcZ0FRnLKBmxkTHwQsbACxnLKBmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGQMvZAy8kLGMklnGGJ/GGB+6Q/4yflwul6dbn8Q9WW2328/r9frx1idyb87n89dlWb4I7vlWM7T9fv+02Wy+v+B579rpdPp4OBwej8fjXBHE9kyr+TBD2+123577JP4xbz94AR8QyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYaHd9zC0z3SHvn+t1ndXcBDj3Ztky8zLzus3durc+j3tiW/j1bAt/IdNVZHxAICM2MmIjIzYyYiMjNjJiIyM2MmJjVP4G1Yi+mEOLBdsAAAAASUVORK5CYII="">","10,760(47.8%)"
3,legal_name[object],"1. nan2. STARBUCKS CORPORATION3. FAMILY DOLLAR, LLC4. WALGREEN CO.5. COMPASS GROUP USA, INC.6. HOST INTERNATIONAL, INC.7. THE UNIVERSITY OF CHICAGO8. BOND DRUG COMPANY OF ILLINOIS,9. JEWEL FOOD STORES, INC.10. POTBELLY SANDWICH WORKS, LLC11. other","10,760 (47.8%)125 (0.6%)69 (0.3%)64 (0.3%)61 (0.3%)49 (0.2%)48 (0.2%)46 (0.2%)43 (0.2%)41 (0.2%)11,198 (49.8%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjMsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvZiW1igAAAAlwSFlzAAAPYQAAD2EBqD+naQAAA/BJREFUeJzt3EtuWlkUQNH7LJRPB2QhMYSqOTCIDDaDYBA1BxRBI06lQ+lKaUcxLm9CvFYDuWO/J7TNAeseL5fLZUDhIbkKiI2S2MiIjYzYyIiNjNjIiI2M2MgsY4yPY4x33SXv1vfL5fJ065u4Z6vtdvtpvV4/3vpGfnfn8/nLsiyfBXe91Qxtv98/bTabby/4OX+00+n04XA4PB6PxzkBxHal1XyYoe12u6/X/pA3Yr7d4AV8QCAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNtojRvO8VnfJ++P5+X+s5gnUeTDQea2fm8/TPBp+6/u4Z3YQfp0dhBda/Mss6le24beWZLtqfmFziNf2MLerfqzyed/Gq/J3NjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM22oWXH19b5uBVORZOxsILGe/ZyDwsy/L3siw2vkle2f4aY7x//Uvx1hmjZMRGRmx

## Check for duplicate rows & columns

In [38]:
duplicates = combined_df[combined_df.duplicated(keep=False)]
duplicates


,license_id,doing_business_as_name,legal_name,business_activity_id,license_description,business_activity,latitude,longitude,city,address,zip_code,location,food_category,location_score,is_food,is_bus_stop,systemstop,street,cross_st,public_nam
